# Letta Python Client 
Welcome to the Letta tutorial! In this tutorial, we'll go through how to create a basic user-client for Letta and create a custom agent with long term memory. 

Letta runs *agents-as-a-service*, so agents can run independently on a server. For this tutorial, we will be connecting to an existing Letta server via the Python client and the UI console. If you don't have a running server, see the [documentation](https://letta.readme.io/docs/running-a-letta-server) for instructions on how to create one.  

## Part 0: Install Letta 

In [ ]:
!pip install git+https://github.com/cpacker/MemGPT.git@tutorials

## Part 1: Connecting to the Letta Client 

The Letta client connects to a running Letta service, specified by `base_url`. The client corresponds to a *single-user* (you), so requires an authentication token to let the service know who you are. 



In [19]:
from letta import create_client

base_url = "http://35.238.125.250:8083"

# TODO: replace with your token 
my_token = "sk-..." 

client = create_client(base_url=base_url, token=my_token) 

### Viewing the developer portal 
Letta provides a portal interface for viewing and interacting with agents, data sources, tools, and more. You can enter `http://35.238.125.250:8083` into your browser to load the developer portal, and enter in `my_token` to log in. 

<img src="./developer_portal_login.png" width="800">

## Part 2: Create an agent 
We'll first start with creating a basic Letta agent. 

In [ ]:
basic_agent = client.create_agent(
    name="basic_agent", 
)
print(f"Created agent: {basic_agent.name}")

We can now send messages from the user to the agent by specifying the `agent_id`: 

In [ ]:
from letta.client.utils import pprint 

response = client.user_message(agent_id=basic_agent.id, message="hello") 
pprint(response.messages)

### Chatting in the developer portal 
You can also chat with the agent inside of the developer portal. Try clicking the chat button in the agent view. 

<img src="./dev_portal_agent_chat.png" width="800">

### Adding Personalization
We can now create a more customized agent, but specifying a custom `human` and `persona` field. 
* The *human* specifies the personalization information about the user interacting with the agent 
* The *persona* specifies the behavior and personality of the event

What makes Letta unique is that the starting *persona* and *human* can change over time as the agent gains new information, enabling it to have evolving memory. We'll see an example of this later in the tutorial.

In [ ]:
# TODO: feel free to change the human and person to what you'd like 
persona = \
"""
You are a friendly and helpful agent!
"""

human = \
"""
I am an Accenture consultant with many specializations. My name is Sarah.
"""

custom_agent = client.create_agent(
    name="custom_agent", 
    human=human, 
    persona=persona
)

### Viewing memory 
You can view and edit the agent's memory inside of the developer console. There are two type of memory, *core* and *archival* memory: 
1. Core memory stores short-term memories in the LLM's context 
2. Archival memory stores long term memories in a vector database

In this example, we'll look at how the agent can modify its core memory with new information. To see the agent's memory, click the "Core Memory" section on the developer console. 

<img src="./dev_portal_memory.png" width="800">

### Referencing memory 
Letta agents can customize their responses based on what memories they have stored. Try asking a question that related to the human and persona you provided. 

In [ ]:
response = client.user_message(agent_id=custom_agent.id, message="what do I work as?") 
pprint(response.messages)

### Evolving memory 
Letta agents have long term memory, and can evolve what they store in their memory over time. In the example below, we make a correction to the previously provided information. See how the agent processes this new information. 

In [ ]:
response = client.user_message(agent_id=custom_agent.id, message="Actually, my name is Charles") 
pprint(response.messages)

Now, look back at the developer portal and at the agent's *core memory*. Do you see a change in the *human* section of the memory? 

## Part 3: Adding Tools 
Letta agents can be connected to custom tools. Currently, tools must be created by service administrators. However, you can add additional tools provided by the service administrator to the agent you create. 

### View Available Tools 

In [ ]:
tools = client.list_tools().tools
for tool in tools: 
    print(f"Tool: {tool.name} - {tool.json_schema['description']}")

### Create a tool using agent in the developer portal 
Create an agent in the developer portal and toggle additional tools you want the agent to use. We recommend modifying the *persona* to notify the agent that it should be using the tools for certain tasks. 


<img src="./dev_portal_tools.png" width="800">

## Part 4: Cleanup (optional) 
You can cleanup the agents you creating the following command to delete your agents: 

In [ ]:
for agent in client.list_agents().agents: 
    client.delete_agent(agent["id"])
    print(f"Deleted agent {agent['name']} with ID {agent['id']}")

## 🎉 Congrats, you're done with day 1 of Letta! 
For day 2, we'll go over how to connect *data sources* to Letta to run RAG agents. 